<a href="https://colab.research.google.com/github/PatrickRogg/machine-learning/blob/main/SpaceInvaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-rl2

In [2]:
import gym 
import random

In [3]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [4]:
env.unwrapped.get_action_meanings()


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:50.0
Episode:2 Score:105.0
Episode:3 Score:55.0
Episode:4 Score:410.0
Episode:5 Score:425.0


In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [33]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [34]:
model = build_model(height, width, channels, actions)


In [35]:
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               34603520  
_________________________________________________________________
dense_14 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_15 (Dense)             (None, 6)                

In [36]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [40]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [41]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [ ]:
dqn.fit(env, nb_steps=1000000, visualize=False, verbose=2)

Training for 1000000 steps ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    913/1000000: episode: 1, duration: 3.516s, episode steps: 913, steps per second: 260, episode reward: 230.000, mean reward:  0.252 [ 0.000, 30.000], mean action: 2.498 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   1385/1000000: episode: 2, duration: 28.665s, episode steps: 472, steps per second:  16, episode reward: 85.000, mean reward:  0.180 [ 0.000, 20.000], mean action: 2.614 [0.000, 5.000],  loss: 1.872351, mean_q: 8.304088, mean_eps: 0.892675
   2076/1000000: episode: 3, duration: 49.725s, episode steps: 691, steps per second:  14, episode reward: 210.000, mean reward:  0.304 [ 0.000, 30.000], mean action: 2.512 [0.000, 5.000],  loss: 0.600249, mean_q: 8.069609, mean_eps: 0.844300
   2899/1000000: episode: 4, duration: 59.329s, episode steps: 823, steps per second:  14, episode reward: 155.000, mean reward:  0.188 [ 0.000, 30.000], mean action: 2.505 [0.000, 5.000],  loss: 0.301673, mean_q: 8.633022, mean_eps: 0.776170
   3703/1000000: episode: 5, duration: 56.829s, episode steps: 804, steps per second:  14, episode reward: 215.000, mean reward:  0.267 [ 0.000, 30.000], mean action: 2.611 [0.000, 5.000],  loss: 0.349457, mean_q: 8.049285, mean_eps: 0.702955
   4676/1000000: episode: 6, 

In [ ]:
dqn.save_weights('SavedWeights/1m/dqn_weights.h5f')